In [1]:
# Necessary imports
import nltk
import glob
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
pos_files = glob.glob("data/clean_positive_train_*.csv")
neg_files = glob.glob("data/clean_negative_train_*.csv")

df_pos_list = [pd.read_csv(open(fp, 'r'), encoding='utf-8', engine='c') for fp in pos_files[:10]]
df_neg_list = [pd.read_csv(open(fp, 'r'), encoding='utf-8', engine='c') for fp in neg_files[:10]]

In [3]:
df = [pd.concat([df_pos, df_neg]) for (df_pos, df_neg) in zip(df_pos_list, df_neg_list)]
df = [single_df.sample(frac=1).reset_index(drop=True) for single_df in df]
print(len(df))

# Now we have scrambeled dataframes of 20000 entries, with features such as binary scores and profanity
df[0].describe()

10


,score,ups,controversiality,parent_score,parent_ups,parent_controversiality,sentiment,profanity,profanity_prob
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,2.000000e+04
mean,0.500000,32.500000,0.001000,0.940550,123.437950,0.002100,0.036857,0.144450,2.213078e-01
std,0.500013,41.501038,0.031608,0.236471,273.303875,0.045779,0.274485,0.351554,2.556282e-01
min,0.000000,-9.000000,0.000000,0.000000,-342.000000,0.000000,-1.000000,0.000000,1.092762e-20
25%,0.000000,-9.000000,0.000000,1.000000,10.000000,0.000000,-0.025000,0.000000,7.562152e-02
50%,0.500000,32.500000,0.000000,1.000000,42.000000,0.000000,0.000000,0.000000,1.199541e-01
75%,1.000000,74.000000,0.000000,1.000000,129.250000,0.000000,0.150000,0.000000,2.229088e-01
max,1.000000,74.000000,1.000000,1.000000,5295.000000,1.000000,1.000000,1.000000,1.000000e+00


In [4]:
df[0].head()

,text,score,ups,controversiality,parent_text,parent_score,parent_ups,parent_controversiality,sentiment,profanity,profanity_prob
0,takes surprising amount energy heat 155 gallon...,1,74,0,nothing like climbing 14000 foot mountain gett...,1,178,0,0.233333,0,0.041541
1,long doesnt slit throat wife bake pie cool wha...,1,74,0,everyone agrees gives good cut right end worth,1,48,0,0.150000,0,0.197282
2,sadly upset diggpatriots posts get downvoted s...,0,-9,0,brilliant looking seinfeld reference disappoin...,1,1,0,-0.250000,0,0.062276
3,randomly come across cp thread wont get arrest...,1,74,0,answered past 4chan asked fbi law enforcement ...,1,297,0,-0.107143,0,0.059548
4,switch different brand like neutrogenas,1,74,0,eye makeup remover burns like hell gets eyes b...,1,197,0,0.000000,0,0.118756


Now that we have a small part of clean text we can start to learn the texts to our machine

In [44]:
# Second we split our data for training and testing
text_all = pd.Series()
for n in range(len(df)):
    text_all = text_all.append(df[n]['text'], ignore_index=True)
text_all = text_all.astype(str)
print(type(text_all[0]))
    
text_data_full = [df[n]['text'].astype(str) for n in range(len(df))]
text_score_full = [df[n]['score'] for n in range(len(df))]
# parent_text_data = df['parent_text']
# parent_text_score = df['parent_score']

<class 'str'>


In [61]:
# We create a tokenizer which will give a word_index integer value to each word
tokenizer = Tokenizer(num_words=10000, lower=True, split=' ', document_count=0)

# Create the word_index list based on all our data
tokenizer.fit_on_texts(text_all)
# Now we make a list of sequences of integers based on our texts
text_vec = []
text_data = []
for n in range(len(df)):
    text_vec.append(tokenizer.texts_to_sequences(df[n]['text'].astype(str)))
    text_data.append(keras.preprocessing.sequence.pad_sequences(text_vec[n], value=0, padding='post', maxlen=128))
print(text_data[0].shape)

(19749, 128)


In [66]:
feature_data = [np.array([])]
score_data = [np.array([])]

X_train = []
X_test = []
y_train = []
y_test = []

for n in range(len(df)):
    cur_df = df[n]
    print(n)
    df[n].dropna(axis=0, inplace=True)
    size = len(cur_df)
    # Iterate over each row in the DataFrame
    f_data = np.array([])
    s_data = np.array([])
    for index, row in cur_df.iterrows():
        
        #All features we use
        sentiment = row['sentiment']
        profanity = row['profanity_prob']
        
        # Stack the features
        features = np.hstack((sentiment, profanity))
        
        if index == 0:
            f_data = features
            s_data = row['score']
        else:
            f_data = np.vstack([f_data, features])
            s_data = np.vstack([s_data, row['score']])
    if n == 0:
        feature_data[0] = f_data
        score_data[0] = s_data
    else:
        feature_data.append(f_data)
        score_data.append(s_data)

    
       

0


ValueError: Found input variables with inconsistent numbers of samples: [0, 19749]

In [67]:
x_all = np.hstack((text_data[0],feature_data[0]))
print(x_all.shape)
X_train, X_test, y_train, y_test = train_test_split(x_all, score_data[0], test_size=0.20, random_state=42)

(19749, 130)


In [32]:
print(len(train_data[0]), len(train_data[1]))
print(max(train_data, key=len))
print(max(test_data, key=len))
print(len(tokenizer.word_index) + 1)

128 128
[263   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
[ 175 1103   22   47   33  818  348  709  299 2061   29   88   38   90
   28   48  212    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0

## Finally we can do some neural networks!

In [33]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

model = tf.keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(24, activation=tf.nn.relu))
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

132620
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          2121920   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                400       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 2,123,017
Trainable params: 2,123,017
Non-trainab

In [34]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [35]:
history = model.fit(train_data,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(test_data, y_test),
                    verbose=1)

Train on 16000 samples, validate on 4000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/40
16000/16000 [==============================] - 1s 57us/sample - loss: 0.6931 - acc: 0.5099 - val_loss: 0.6930 - val_acc: 0.5015
Epoch 2/40
16000/16000 [==============================] - 1s 38us/sample - loss: 0.6927 - acc: 0.5104 - val_loss: 0.6924 - val_acc: 0.5015
Epoch 3/40
16000/16000 [==============================] - 1s 38us/sample - loss: 0.6911 - acc: 0.5658 - val_loss: 0.6900 - val_acc: 0.5832
Epoch 4/40
16000/16000 [==============================] - 1s 41us/sample - loss: 0.6868 - acc: 0.5811 - val_loss: 0.6874 - val_acc: 0.5010
Epoch 5/40
16000/16000 [==============================] - 1s 38us/sample - loss: 0.6768 - acc: 0.5965 - val_loss: 0.6773 - val_acc: 0.6018
Epoch 6/40
16000/16000 [==============================] - 1s 38us/sample - loss: 0.6575 - acc: 0.6397 - val_loss: 0.6656 - val_acc: 0.5962
Epoch 7/40
16000/16000 [==============================] - 1s 38us/sam